In [6]:
import sys
import os 

paths = ['../src/', '../src/api/v6']
for path in paths:
    path = os.path.normcase(path)
    if not any(os.path.normcase(sp) == path for sp in sys.path):
        sys.path.append(path)

In [7]:
sys.path

['/Users/asaf/Workspace/biu/complex-utterance-to-code/notebooks',
 '/Users/asaf/Workspace/biu/complex-utterance-to-code',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python39.zip',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python3.9',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python3.9/lib-dynload',
 '',
 '/Users/asaf/.local/lib/python3.9/site-packages',
 '/Users/asaf/opt/miniconda3/envs/biu/lib/python3.9/site-packages',
 '../src/',
 '../src/api/v6']

In [8]:
text = """
    [ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ 
						[ recipient1, recipient2 ]
					]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]
 """

In [9]:
import ast
from representations.tree.tree import Tree
from representations.builders.ast.tearers.tearer_factory import TearerFactory

In [10]:
tree = Tree.unparse(text)
print(tree)

	[ Module
		[ source = Location.resolve_from_text('Walmart') ]
		[ destination = Location.resolve_from_text('Walgreens') ]
		[ directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time) ]
		[ Responder.respond(response=directions) ]
		[ recipient1 = Contact.resolve_from_text('Pepsi') ]
		[ recipient2 = Contact.resolve_from_text('Walgreens') ]
		[ For
			[ test
				[ iter
					[ [ recipient1, recipient2 ] ]
				]
				[ Name
					[ recipient ]
				]
			]
			[ body
				[ exact_content = Content.resolve_from_text('check it out') ]
				[ Message.send_message(recipient=recipient, exact_content=exact_content) ]
			]
		]
	]



In [11]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

source = Location.resolve_from_text('Walmart')
destination = Location.resolve_from_text('Walgreens')
directions = Navigation.find_directions(source=source, destination=destination, date_time=date_time)
Responder.respond(response=directions)
recipient1 = Contact.resolve_from_text('Pepsi')
recipient2 = Contact.resolve_from_text('Walgreens')
for recipient in [recipient1, recipient2]:
    exact_content = Content.resolve_from_text('check it out')
    Message.send_message(recipient=recipient, exact_content=exact_content)


In [12]:
import pandas as pd

df = pd.read_csv('../build/eval_complex_utterance_to_code_with_intermediate_61_20230509.csv.gz')

In [13]:
df.iloc[0]['text']

'Check the availability of Pepsi at Walmart and also check it at Walgreens.'

In [14]:
df.iloc[0]['code_rep']

"[ Module [ product_name = ProductName.resolve_from_text('Pepsi') ] [ location = Location.resolve_from_text('Walmart') ] [ products = Shopping.find_products(product_name=product_name, location=location) ] [ Responder.respond(response=products) ] [ location = Location.resolve_from_text('Walgreens') ] [ products = Shopping.find_products(product_name=product_name, location=location) ] [ Responder.respond(response=products) ] ]"

In [15]:
tree = Tree.unparse(df.iloc[0]['code_rep'])

In [16]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

product_name = ProductName.resolve_from_text('Pepsi')
location = Location.resolve_from_text('Walmart')
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)
location = Location.resolve_from_text('Walgreens')
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)


In [17]:
import utils.api_utils

In [18]:
for row in df.iterrows():
    print(row[1]['text'])

Check the availability of Pepsi at Walmart and also check it at Walgreens.
If it's raining tomorrow morning, set my alarm for 7:30, if it's not, set my alarm for 8.
Play the new Taylor Swift album and pull up my shopping list for today.
Send a message to dad if it rains tomorrow.
Give me directions to Navy pier in Chicago and tell me what the current traffic is looking like.
Check the next time Blink 182 will be in Chicago and tell me the ticket prices.
Remind me tomorrow to email Jim about lunch and schedule a reservation for noon at the cafe.
Can you place an order for two turkeys to arrive the 22nd, and remind me about it on the 21st?
How long will it take to get to AMC theater at 8pm tonight and tell me what tonight's weather outlook is.
Check the weather for the 4th of July and send a text to Grandpa to invite him over and tell him the weather.
Set a timer for one hour and text Stacy that dinner will be ready in one hour.
If the weather is cold tomorrow please remind me to grab my

In [19]:
text = df.iloc[-1]['code_rep']
text

"[ Module [ event_names = EventName.resolve_many_from_text('free events') ] [ date_times = DateTime.resolve_many_from_text('this weekend') ] [ events = [] ] [ For [ test [ iter [ event_names ] ] [ Name [ event_name ] ] ] [ body [ For [ test [ iter [ date_times ] ] [ Name [ date_time ] ] ] [ body [ events += Calendar.find_events(event_name=event_name, date_time=date_time) ] ] ] ] ] [ Responder.respond(response=events) ] [ weather_forecasts = Weather.find_weather_forecasts() ] [ Responder.respond(response=weather_forecasts) ] ]"

In [20]:
print(df.iloc[-1]['code'])

event_names = EventName.resolve_many_from_text("free events")
date_times = DateTime.resolve_many_from_text("this weekend")
events = []
for event_name in event_names:
    for date_time in date_times:
        events += Calendar.find_events(event_name=event_name, date_time=date_time)
Responder.respond(response=events)

weather_forecasts = Weather.find_weather_forecasts()
Responder.respond(response=weather_forecasts)



In [21]:
tree = Tree.unparse(text)#.replace('[]', ''))
print(tree)

	[ Module
		[ event_names = EventName.resolve_many_from_text('free events') ]
		[ date_times = DateTime.resolve_many_from_text('this weekend') ]
		[ events = [] ]
		[ For
			[ test
				[ iter
					[ event_names ]
				]
				[ Name
					[ event_name ]
				]
			]
			[ body
				[ For
					[ test
						[ iter
							[ date_times ]
						]
						[ Name
							[ date_time ]
						]
					]
					[ body
						[ events += Calendar.find_events(event_name=event_name, date_time=date_time) ]
					]
				]
			]
		]
		[ Responder.respond(response=events) ]
		[ weather_forecasts = Weather.find_weather_forecasts() ]
		[ Responder.respond(response=weather_forecasts) ]
	]



In [23]:
tearer = TearerFactory().get_tearer(tree.root_node)
asdl2 = tearer.tear(tree.root_node)
x = ast.unparse(asdl2)
print(x)

event_names = EventName.resolve_many_from_text('free events')
date_times = DateTime.resolve_many_from_text('this weekend')
events = []
for event_name in event_names:
    for date_time in date_times:
        events += Calendar.find_events(event_name=event_name, date_time=date_time)
Responder.respond(response=events)
weather_forecasts = Weather.find_weather_forecasts()
Responder.respond(response=weather_forecasts)


In [60]:
from main_representations import (
    generate_text_representation,
    generate_code_representation,
)

code = df.iloc[-1]['code']

rep_code = generate_code_representation(
    code, rules_enabled=False
)



In [65]:
print(rep_code)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x106248be0>>


TypeError: __repr__ returned non-string (type Node)